# Transition state geometry optimization

Once we have a suitable candidate for transition state we can optimize the structure for a transition state. As explained before the transition state will be characterized by a vanishing gradient and one negative Hessian 
eigenvalue. Therefore, in essence, we are exploring the PES looking for a maximum.

Let us try with a simple example:

### Importing modules

In [8]:
import veloxchem as vlx
import py3Dmol as p3d 

### Visualization of the molecule
First we create a simple function that takes as an input a xyz format coordinates and plot the molecule

In [18]:
#Function to plot geometries
def drawXYZGeom(geom):
    view = p3d.view(width=400, height=400)
    view.addModel(geom, "xyz")
    view.setStyle({'stick':{}})
    view.zoomTo()
    return(view.show())
## Function to compare geometries
def compXYZGeom(init,final):
    view = p3d.view(width=400, height=400)
    view.addModel(init, "xyz")
    view.addModel(final, "xyz")
    view.setStyle({'stick':{}})
    view.zoomTo()
    return(view.show())

Then we just introduce the xyz coordinates as a string and we can plot the structure

In [3]:
#Visualizing the structure
geom_xyz = '''8
Initial geometry
C        1.4059000000    0.8128000000    0.0009000000
C        1.5654000000   -0.3926000000    0.0009000000
N       -1.0383000000    1.0196000000   -0.0052000000
H       -2.0386000000    1.4291000000    0.0210000000
H        1.3741000000    1.8940000000   -0.0007000000
H        1.9546000000   -1.4028000000    0.0003000000
N       -0.4958000000   -1.3375000000   -0.0011000000
N       -1.1970000000   -0.3166000000    0.0018000000
'''
drawXYZGeom(geom_xyz)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Definition of the geometry and setting up the SCF driver

In [ ]:
## Finding the TS
geometry = vlx.Molecule.from_xyz_string(geom_xyz)
basis = vlx.MolecularBasis.read(geometry,"sto-3g")
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.compute(geometry,basis)

### Setting up the gradient driver

In [5]:
scf_grad_drv = vlx.ScfGradientDriver(scf_drv)
gradient_settings = {'numerical:no'}
scf_grad_drv.update_settings(gradient_settings,method_dict={'xcfun: b3lyp'})

### Setting up the optimization driver

In [ ]:
scf_opt_drv = vlx.OptimizationDriver(scf_grad_drv,flag='scf')
scf_opt_drv.transition = True
scf_opt_drv.hessian = 'first+last'
scf_opt_drv.compute(geometry,basis)

The output will give four files: <br />
**Geometries during the optimization in xyz format (_optim.xyz)**
```
8    
Iteration 0 Energy -237.48406869
C        1.4059000000    0.8128000000    0.0009000000
C        1.5654000000   -0.3926000000    0.0009000000
N       -1.0383000000    1.0196000000   -0.0052000000
H       -2.0386000000    1.4291000000    0.0210000000
H        1.3741000000    1.8940000000   -0.0007000000
H        1.9546000000   -1.4028000000    0.0003000000
N       -0.4958000000   -1.3375000000   -0.0011000000
N       -1.1970000000   -0.3166000000    0.0018000000
8    
Iteration 1 Energy -237.48588381
C        1.3986767507    0.8076140878    0.0009002477
C        1.5554932198   -0.3912703043    0.0008560542
N       -1.0339417937    1.0131015966   -0.0050309571
H       -2.0296875775    1.4272388306    0.0208045784
H        1.3724942305    1.8878930527   -0.0007271680
H        1.9523159399   -1.3970927549    0.0002038018
N       -0.4855076024   -1.3246755992   -0.0010344286
N       -1.1995439420   -0.3168060274    0.0019270504
8    
Iteration 2 Energy -237.48754838
C        1.3881987993    0.8008460851    0.0008959673
C        1.5414491982   -0.3899861931    0.0007853320
N       -1.0276095642    1.0039979473   -0.0047832956
H       -2.0164700299    1.4244073465    0.0205164352
H        1.3697024299    1.8795054853   -0.0007658631
H        1.9489162104   -1.3892473489    0.0000581691
N       -0.4711152364   -1.3064025713   -0.0009230192
N       -1.2027737500   -0.3171424570    0.0021141842
8    
.
.
.
Iteration 8 Energy -237.48730935
C        1.2694737152    0.8199118237    0.0020300631
C        1.4650268055   -0.3557331487   -0.0014081365
N       -0.9624886689    0.9369706825   -0.0070753368
H       -1.9114607569    1.3882275308    0.0242133169
H        1.2776270348    1.8879026382    0.0037766854
H        1.9579288423   -1.3019609852   -0.0042985096
N       -0.3994930133   -1.3005046839   -0.0013522025
N       -1.1661868246   -0.3716732596    0.0018726559
8    
Iteration 9 Energy -237.48730981
C        1.2687990140    0.8198482259    0.0024123532
C        1.4652286829   -0.3556615967   -0.0016439073
N       -0.9625706742    0.9373517016   -0.0080640143
H       -1.9119100993    1.3873539839    0.0265832651
H        1.2768799142    1.8879461569    0.0046771752
H        1.9591653948   -1.3012461760   -0.0048335460
N       -0.3997144523   -1.3008164821   -0.0025853684
N       -1.1654403884   -0.3717101556    0.0012082555
```
**A log file with the output from Geometric** <br />
<br />
**The initial and final hessian with the Gibbs free energy** <br />
The last will look like this:
```
# == Summary of harmonic free energy analysis ==
# Note: Rotational symmetry is set to 1 regardless of true symmetry
# Note: Free energy does not include contribution from 2 imaginary mode(s)
# 
# Gibbs free energy contributions calculated at @ 300.00 K:
# Zero-point vibrational energy:                                   35.1393 kcal/mol 
# H   (Trans + Rot + Vib = Tot):   1.4904 +   0.8942 +   0.9940 =   3.3787 kcal/mol 
# S   (Trans + Rot + Vib = Tot):  38.6717 +  24.9750 +   4.9343 =  68.5810 cal/mol/K
# TS  (Trans + Rot + Vib = Tot):  11.6015 +   7.4925 +   1.4803 =  20.5743 kcal/mol 
# 
# Ground State Electronic Energy    : E0                        =  -237.48730981 au (   -149025.5369 kcal/mol)
# Free Energy Correction (Harmonic) : ZPVE + [H-TS]_T,R,V       =     0.02859519 au (        17.9438 kcal/mol)
# Gibbs Free Energy (Harmonic)      : E0 + ZPVE + [H-TS]_T,R,V  =  -237.45871462 au (   -149007.5931 kcal/mol)
# 

8
Iteration 9 Energy -237.48730981 (Optimized Structure)
 C    1.2687990140    0.8198482259    0.0024123532
 C    1.4652286829   -0.3556615967   -0.0016439073
 N   -0.9625706742    0.9373517016   -0.0080640143
 H   -1.9119100993    1.3873539839    0.0265832651
 H    1.2768799142    1.8879461569    0.0046771752
 H    1.9591653948   -1.3012461760   -0.0048335460
 N   -0.3997144523   -1.3008164821   -0.0025853684
 N   -1.1654403884   -0.3717101556    0.0012082555

```


In [19]:
#Comparision initial and final structure
final = '''8
Iteration 9 Energy -237.48730981
C        1.2687990140    0.8198482259    0.0024123532
C        1.4652286829   -0.3556615967   -0.0016439073
N       -0.9625706742    0.9373517016   -0.0080640143
H       -1.9119100993    1.3873539839    0.0265832651
H        1.2768799142    1.8879461569    0.0046771752
H        1.9591653948   -1.3012461760   -0.0048335460
N       -0.3997144523   -1.3008164821   -0.0025853684
N       -1.1654403884   -0.3717101556    0.0012082555
'''
compXYZGeom(geom_xyz,final)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Calculation of the activation free energy 
By definition the activation free energy is $\Delta G^{\ddag} = G^{\ddag} - G_{react}$. So we just need to get the free energy of the reactant. Since we have observed that the candidate to TS that we have optimized before is further away than the TS, minimizing that structure will lead to the reactant.

In [ ]:
## Going to reactant by minimizing the structure
scf_opt_drv = vlx.OptimizationDriver(scf_grad_drv,flag='scf')
scf_opt_drv.transition = False ## Will minimize the energy
scf_opt_drv.hessian = 'last'
scf_opt_drv.compute(geometry,basis)

### Comparing reactant and product 

In [21]:
reactant = '''8
Iteration 42 Energy -237.52638154 (Optimized Structure)
 C    1.9112351040    0.1624677572   -0.0006980640
 C    3.0548281772   -0.0782577796    0.0004981290
 N   -1.3015267488    0.8281271773    0.0008357891
 H   -1.8259069457    1.7432222546    0.0231535592
 H    0.8632242900    0.3826709271   -0.0017612332
 H    4.0964503076   -0.2975291731    0.0015528206
 N   -2.9848455585   -0.9798798745   -0.0056716903
 N   -2.2831586608   -0.0548212877   -0.0000093104
'''
compXYZGeom(final,reactant)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Indeed, we can see how see that in the reactant the molecules are further away. <br />
From the outputs of the TS and reactant we can get the activation free energy $\Delta G^{\ddag}$ that we can relate with TST to estimate the rate of the reaction.<br />
<br />
From the **vdata_last** files we can get the free energy: <br />
```
Gibbs Free Energy TS (Harmonic)      : E0 + ZPVE + [H-TS]_T,R,V  =  -237.45871462 au (   -149007.5931 kcal/mol)
Gibbs Free Energy Reactant (Harmonic)      : E0 + ZPVE + [H-TS]_T,R,V  =  -237.50434988 au (   -149036.2297 kcal/mol)
```
So, the activation free energy will be $\Delta G^{\ddag} = G^{\ddag} - G_{react}$

In [25]:
g_ts = -149007.5931
g_reac = -149036.2297
delta_g = g_ts-g_reac
print('Activation Free Energy in kcal/mol is:',"%.2f" % delta_g)

Activation Free Energy in Kcal/mol is: 28.64


Applying Transition State Theory we can estimate the rate with the Eyring equation: <br />
$$ k = \frac{k_B T}{h}exp(\frac{-\Delta G^{\ddag}}{RT}) $$
R in kcal/mol K is 1.958e-3 <br />
T is 300 K<br />
kb in J/K is 1.3806e-23<br />
h in J s is 6.6261e-34<br />

In [26]:
import numpy as np
def rate(delta_g):
    k = (1.3806e-23*300/6.6261e-34)* np.exp(-delta_g/(1.958e-3*300))
    return k

In [32]:
print('The reaction rate is',"%10.3e" % rate(delta_g),'s-1')

The reaction rate is  4.202e-09 s-1
